<a href="https://colab.research.google.com/github/Sammy2414/BuffaloPrep/blob/main/Team4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download devansodariya/student-performance-data

  0% 0.00/7.16k [00:00<?, ?B/s]
100% 7.16k/7.16k [00:00<00:00, 13.1MB/s]


In [ ]:
! unzip student-performance-data.zip

Archive:  student-performance-data.zip
  inflating: student_data.csv        


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
studentData = pd.read_csv("student_data")

In [ ]:
data = pd.read_csv("buffaloPrep.csv")
data

,Timestamp,School,Size of Family,Mother's Education,Father's Education,Reason to choose Buffalo Prep,Home to school travel time,Weekly study time,Number of past class failures,Do you receive extra educational support,Do you receive family educational support,Do you participate in extra-curricular activities,Internet access at home,Free time after school,Current Health Status,Number of Absences,Math Competency,Reading Competency,Writing Competency
0,7/6/2023 16:43:50,Public,Less than or equal to 3,Higher Education,Secondary Education,Close to home,15 to 30 minutes,2 to 5 hours,1,No,No,No,Yes,Normal,Good,23,Novice,Intermediate,Novice
1,7/7/2023 9:20:19,Public,Greater than 3,None,None,Reputation,More than an hour,more than 10 hours,4+,No,No,No,No,Very High,Very Good,10,Master,Master,Distinguished
2,7/7/2023 9:20:47,Private,Less than or equal to 3,None,Primary Education (4th grade),Close to home,30 to 1 hour,1 to 2 hours,1,Yes,No,Yes,Yes,Normal,Very Good,0,Master,Proficient,Master
3,7/7/2023 9:21:34,Private,Greater than 3,None,None,Reputation,1 to 15 minutes,1 to 2 hours,0,No,No,No,Yes,Very High,Good,2,Master,Proficient,Intermediate
4,7/7/2023 9:21:36,Public,Greater than 3,None,None,Close to home,More than an hour,2 to 5 hours,0,No,Yes,Yes,Yes,Low,Good,23,Master,Proficient,Master
5,7/7/2023 9:21:53,Private,Greater than 3,Higher Education,Higher Education,Course preference,More than an hour,5 to 10 hours,0,No,No,Yes,Yes,Low,Good,3,Proficient,Proficient,Distinguished
6,7/7/2023 9:25:24,Public,Less than or equal to 3,Secondary Education,5th to 9th Grade,Close to home,30 to 1 hour,2 to 5 hours,4+,No,Yes,No,Yes,Low,Average,7,Intermediate,Novice,Proficient
7,7/7/2023 9:26:08,Public,Less than or equal to 3,Higher Education,Higher Education,Reputation,1 to 15 minutes,1 to 2 hours,0,Yes,Yes,Yes,Yes,Very Low,Average,0,Proficient,Proficient,Proficient
8,7/7/2023 9:27:17,Private,Less than or equal to 3,5th to 9th Grade,Primary Education (4th grade),Reputation,1 to 15 minutes,1 to 2 hours,4+,No,No,No,No,Very High,Very Good,30,Novice,Novice,Intermediate


In [ ]:
gradeDict = {'Master' : 20, 'Distinguished': 16, 'Proficient' : 12, 'Intermediate': 8, 'Novice': 4}
schoolDict = {'Public': 0, 'Private': 1}
famSizeDict = {'Less than or equal to 3': 0, 'Greater than 3': 1}
educationDict = {'None' : 0, 'Primary Education (4th grade)': 1, '5th to 9th Grade': 2, 'Secondary Education': 3, 'Higher Education': 4}
travelTimeDict = {'1 to 15 minutes' : 0, '15 to 30 minutes': 1, '30 to 1 hour': 2, 'More than an hour': 3}
freeTimeDict = {'Very High' : 4, 'High': 3, 'Normal' : 2, 'Low': 1, 'Very Low': 0}
binaryDict = {'Yes' : 1, 'No': 0}
reasonDict = {'Close to home' : 3, 'Reputation': 2, 'Course preference' : 1, 'Other': 0}
weeklyStudyDict = {'1 to 2 hours' : 3, '2 to 5 hours': 2, '5 to 10 hours' : 1, 'more than 10 hours': 0}
currentHealthDict = {'Very Good' : 4, 'Good': 3, 'Average' : 2, 'Low': 1, 'Very Low': 0}

data['Math Competency'] = data['Math Competency'].map(gradeDict)
data['Reading Competency'] = data['Reading Competency'].map(gradeDict)
data['Writing Competency'] = data['Writing Competency'].map(gradeDict)
data['School'] = data['School'].map(schoolDict)
data['Size of Family'] = data['Size of Family'].map(famSizeDict)
data['Mother\'s Education'] = data['Mother\'s Education'].map(educationDict)
data['Father\'s Education'] = data['Father\'s Education'].map(educationDict)
data['Home to school travel time'] = data['Home to school travel time'].map(travelTimeDict)
data['Free time after school'] = data['Free time after school'].map(freeTimeDict)
data['Do you receive extra educational support'] = data['Do you receive extra educational support'].map(binaryDict)
data['Do you receive family educational support'] = data['Do you receive family educational support'].map(binaryDict)
data['Do you participate in extra-curricular activities'] = data['Do you participate in extra-curricular activities'].map(binaryDict)
data['Internet access at home'] = data['Internet access at home'].map(binaryDict)
data['Number of past class failures'] = data['Number of past class failures'].apply(lambda x: int(x.rstrip('+')))
data['Reason to choose Buffalo Prep'] = data['Reason to choose Buffalo Prep'].map(reasonDict)
data['Weekly study time'] = data['Weekly study time'].map(weeklyStudyDict)
data['Current Health Status'] = data['Current Health Status'].map(currentHealthDict)

data

,Timestamp,School,Size of Family,Mother's Education,Father's Education,Reason to choose Buffalo Prep,Home to school travel time,Weekly study time,Number of past class failures,Do you receive extra educational support,Do you receive family educational support,Do you participate in extra-curricular activities,Internet access at home,Free time after school,Current Health Status,Number of Absences,Math Competency,Reading Competency,Writing Competency
0,7/6/2023 16:43:50,0,0,4,3,3,1,2,1,0,0,0,1,2,3,23,4,8,4
1,7/7/2023 9:20:19,0,1,0,0,2,3,0,4,0,0,0,0,4,4,10,20,20,16
2,7/7/2023 9:20:47,1,0,0,1,3,2,3,1,1,0,1,1,2,4,0,20,12,20
3,7/7/2023 9:21:34,1,1,0,0,2,0,3,0,0,0,0,1,4,3,2,20,12,8
4,7/7/2023 9:21:36,0,1,0,0,3,3,2,0,0,1,1,1,1,3,23,20,12,20
5,7/7/2023 9:21:53,1,1,4,4,1,3,1,0,0,0,1,1,1,3,3,12,12,16
6,7/7/2023 9:25:24,0,0,3,2,3,2,2,4,0,1,0,1,1,2,7,8,4,12
7,7/7/2023 9:26:08,0,0,4,4,2,0,3,0,1,1,1,1,0,2,0,12,12,12
8,7/7/2023 9:27:17,1,0,2,1,2,0,3,4,0,0,0,0,4,4,30,4,4,8


In [ ]:
# encoder = LabelEncoder()

# for column in range(len(data.columns)):
#   data[data.columns[column]] = encoder.fit_transform(data[data.columns[column]])

,Timestamp,School,Size of Family,Mother's Education,Father's Education,Reason to choose this school,Home to school travel time,Weekly study time,Number of past class failures,Do you receive extra educational support,...,Do you participate in extra-curricular activities,Internet access at home,Free time after school,Current Health Status,Number of Absences,Math Competency,English Proficiency (0-20),Unnamed: 18,Reading Competency,Writing Competency
0,0,1,1,1,4,0,1,1,1,0,...,0,1,1,1,5,4,0,0,3,4
1,1,1,0,2,2,2,3,3,2,0,...,0,0,2,2,4,1,0,0,0,0
2,2,0,1,2,3,0,2,0,1,1,...,1,1,1,2,0,1,0,0,2,2
3,3,0,0,2,2,2,0,0,0,0,...,0,1,2,1,1,1,0,0,2,1
4,4,1,0,2,2,0,3,1,0,0,...,1,1,0,1,5,1,0,0,2,2
5,5,0,0,1,1,1,3,2,0,0,...,1,1,0,1,2,3,0,0,2,0
6,6,1,1,3,0,0,2,1,2,0,...,0,1,0,0,3,0,0,0,1,3
7,7,1,1,1,1,2,0,0,0,1,...,1,1,3,0,0,3,0,0,2,3
8,8,0,1,0,3,2,0,0,2,0,...,0,0,2,2,6,2,0,0,1,1


In [ ]:
# data['Average'] = data[['G1', 'G2', 'G3']].mean(axis=1)
# data['Average'] = data['Average'].round().astype(int)
data['Average'] = (data[['Math Competency', 'Reading Competency', 'Writing Competency']].mean(axis=1) >= 12).astype(int)
# data['Average'] = (data[['G1', 'G2', 'G3']].mean(axis=1))

# data['Average'] = data['Average'].apply(lambda x: 0 if x < 8 else (1 if x < 12 else 2))


In [ ]:
y = data['Average']
X = data.drop(['Average', 'Timestamp'], axis=1)

In [ ]:
# y = data['Average']
# X = data.drop(['Average', 'G1', 'G2', 'G3', 'Dalc', 'Walc', 'Mjob', 'Fjob', 'goout', 'famrel', 'romantic', 'higher', 'nursery', 'paid', 'guardian', 'Pstatus', 'address', 'age', 'sex'], axis=1)

In [ ]:
y

0    0
1    1
2    1
3    1
4    1
5    1
6    0
7    1
8    0
Name: Average, dtype: int64

In [ ]:
X

,School,Size of Family,Mother's Education,Father's Education,Reason to choose Buffalo Prep,Home to school travel time,Weekly study time,Number of past class failures,Do you receive extra educational support,Do you receive family educational support,Do you participate in extra-curricular activities,Internet access at home,Free time after school,Current Health Status,Number of Absences,Math Competency,Reading Competency,Writing Competency
0,0,0,4,3,3,1,2,1,0,0,0,1,2,3,23,4,8,4
1,0,1,0,0,2,3,0,4,0,0,0,0,4,4,10,20,20,16
2,1,0,0,1,3,2,3,1,1,0,1,1,2,4,0,20,12,20
3,1,1,0,0,2,0,3,0,0,0,0,1,4,3,2,20,12,8
4,0,1,0,0,3,3,2,0,0,1,1,1,1,3,23,20,12,20
5,1,1,4,4,1,3,1,0,0,0,1,1,1,3,3,12,12,16
6,0,0,3,2,3,2,2,4,0,1,0,1,1,2,7,8,4,12
7,0,0,4,4,2,0,3,0,1,1,1,1,0,2,0,12,12,12
8,1,0,2,1,2,0,3,4,0,0,0,0,4,4,30,4,4,8


In [ ]:
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
X

,School,Size of Family,Mother's Education,Father's Education,Reason to choose Buffalo Prep,Home to school travel time,Weekly study time,Number of past class failures,Do you receive extra educational support,Do you receive family educational support,Do you participate in extra-curricular activities,Internet access at home,Free time after school,Current Health Status,Number of Absences,Math Competency,Reading Competency,Writing Competency
0,-0.894427,-0.894427,1.178330,0.852803,1.0,-0.441942,-0.111803,-0.313728,-0.534522,-0.707107,-0.894427,0.534522,-0.076696,-0.150756,1.119732,-1.428869,-0.577350,-1.690309
1,-0.894427,1.118034,-1.054295,-1.066004,-0.5,1.149049,-2.124265,1.380403,-0.534522,-0.707107,-0.894427,-1.870829,1.303840,1.206045,-0.082182,1.020621,2.020726,0.591608
2,1.118034,-0.894427,-1.054295,-0.426401,1.0,0.353553,0.894427,-0.313728,1.870829,-0.707107,1.118034,0.534522,-0.076696,1.206045,-1.006731,1.020621,0.288675,1.352247
3,1.118034,1.118034,-1.054295,-1.066004,-0.5,-1.237437,0.894427,-0.878438,-0.534522,-0.707107,-0.894427,0.534522,1.303840,-0.150756,-0.821821,1.020621,0.288675,-0.929670
4,-0.894427,1.118034,-1.054295,-1.066004,1.0,1.149049,-0.111803,-0.878438,-0.534522,1.414214,1.118034,0.534522,-0.766965,-0.150756,1.119732,1.020621,0.288675,1.352247
5,1.118034,1.118034,1.178330,1.492405,-2.0,1.149049,-1.118034,-0.878438,-0.534522,-0.707107,1.118034,0.534522,-0.766965,-0.150756,-0.729367,-0.204124,0.288675,0.591608
6,-0.894427,-0.894427,0.620174,0.213201,1.0,0.353553,-0.111803,1.380403,-0.534522,1.414214,-0.894427,0.534522,-0.766965,-1.507557,-0.359547,-0.816497,-1.443376,-0.169031
7,-0.894427,-0.894427,1.178330,1.492405,-0.5,-1.237437,0.894427,-0.878438,1.870829,1.414214,1.118034,0.534522,-1.457233,-1.507557,-1.006731,-0.204124,0.288675,-0.169031
8,1.118034,-0.894427,0.062017,-0.426401,-0.5,-1.237437,0.894427,1.380403,-0.534522,-0.707107,-0.894427,-1.870829,1.303840,1.206045,1.766916,-1.428869,-1.443376,-0.929670


In [ ]:
X = X.rename(columns={'School': 'school', 'Size of Family': 'famsize', 'Mother\'s Education': 'Medu', 'Father\'s Education': 'Fedu', 'Reason to choose Buffalo Prep': 'reason', 'Home to school travel time': 'traveltime', 'Weekly study time': 'stufytime', 'Number of past class failures': 'failures', 'Do you receive extra educational support': 'schoolsup', 'Do you receive family educational support': 'famsup', ''})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7) #student_data.csv

log_model = LogisticRegression()
svm_model = SVC(kernel='rbf')
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

log_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
clf.fit(X_train, y_train)

print(f"Logistic Regression: {log_model.score(X_test, y_test)}")
print(f"Support Vector Machine: {svm_model.score(X_test, y_test)}")
print(f"Random Forest: {rf_model.score(X_test, y_test)}")

Logistic Regression: 0.8235294117647058
Support Vector Machine: 0.8235294117647058
Random Forest: 0.8235294117647058


In [ ]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Sequential()

# Add input layer and hidden layers
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))

# Add output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
9/9 [==============================] - 1s 3ms/step - loss: 0.7008 - accuracy: 0.4746
Epoch 2/10
9/9 [==============================] - 0s 3ms/step - loss: 0.5380 - accuracy: 0.8696
Epoch 3/10
9/9 [==============================] - 0s 2ms/step - loss: 0.4441 - accuracy: 0.8732
Epoch 4/10
9/9 [==============================] - 0s 4ms/step - loss: 0.3904 - accuracy: 0.8732
Epoch 5/10
9/9 [==============================] - 0s 4ms/step - loss: 0.3665 - accuracy: 0.8732
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3536 - accuracy: 0.8732
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3439 - accuracy: 0.8732
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3348 - accuracy: 0.8732
Epoch 9/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3265 - accuracy: 0.8732
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3187 - accuracy: 0.8732


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Neural Network Accuracy: {accuracy}")

4/4 [==============================] - 0s 3ms/step - loss: 0.4653 - accuracy: 0.8235
Neural Network Accuracy: 0.8235294222831726


In [ ]:
print(f"Logistic Regression: {log_model.score(X, y)}")
print(f"Support Vector Machine: {svm_model.score(X, y)}")
print(f"Random Forest: {rf_model.score(X, y)}")
loss, accuracy = model.evaluate(X, y)
print(f"Neural Network Accuracy: {accuracy}")

ValueError: ignored